In [ ]:
import wfdb
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.preprocessing import MinMaxScaler

# Load ECG Signal
record_name = "rec_1"
record = wfdb.rdrecord(record_name)
raw_signal = record.p_signal[:, 0]

# Create DataFrame (Only Raw ECG Signal)
df_ecg = pd.DataFrame({
    "Time (ms)": np.arange(len(raw_signal)) * 2,  # 500 Hz → 2ms per sample
    "Raw ECG": raw_signal
})

# **🔹 User Input for Bandpass Filter Parameters**
lowcut = float(input("Enter lower cutoff frequency for bandpass filter (e.g., 0.5): "))
highcut = float(input("Enter higher cutoff frequency for bandpass filter (e.g., 50): "))
fs = 500  # Sampling frequency (Hz)
filter_order = int(input("Enter filter order (e.g., 4): "))

def butter_bandpass(lowcut, highcut, fs, order):
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_bandpass_filter(signal, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order)
    return filtfilt(b, a, signal)

df_ecg["Raw ECG"] = apply_bandpass_filter(df_ecg["Raw ECG"], lowcut, highcut, fs, filter_order)

# **🔹 User Input for Wavelet Transform Parameters**
wavelet_type = input("Enter wavelet type (e.g., db6, coif5, sym4): ")
wavelet_level = int(input("Enter number of decomposition levels (e.g., 3): "))
threshold_factor = float(input("Enter threshold factor for wavelet denoising (e.g., 0.6): "))

def remove_motion_artifacts_wavelet(signal, wavelet, level, threshold_factor):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    coeffs[1:] = [pywt.threshold(c, np.std(c) * threshold_factor, mode="soft") for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)[:len(signal)]

df_ecg["Raw ECG"] = remove_motion_artifacts_wavelet(df_ecg["Raw ECG"], wavelet_type, wavelet_level, threshold_factor)

# **🔹 User Input for Outlier Detection**
iqr_multiplier = float(input("Enter IQR multiplier for outlier removal (e.g., 1.5): "))

Q1 = df_ecg["Raw ECG"].quantile(0.25)
Q3 = df_ecg["Raw ECG"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - iqr_multiplier * IQR
upper_bound = Q3 + iqr_multiplier * IQR

# Remove Outliers
df_ecg = df_ecg[(df_ecg["Raw ECG"] >= lower_bound) & (df_ecg["Raw ECG"] <= upper_bound)]

# **Normalize AFTER Preprocessing**
scaler = MinMaxScaler(feature_range=(0, 1))
df_ecg["ECG Signal"] = scaler.fit_transform(df_ecg[["Raw ECG"]])

# **Drop Unnecessary Columns**
df_ecg = df_ecg.drop(columns=["Raw ECG"])  # Keep only the final ECG Signal

# **🔹 User Input for R-PEAK Detection**
min_distance = int(input("Enter minimum distance between R-peaks (e.g., fs//2.5): "))
min_height = float(input("Enter minimum peak height (e.g., 0.5): "))

peaks, _ = find_peaks(df_ecg["ECG Signal"], distance=min_distance, height=min_height)

# **Calculate RR Intervals**
rr_intervals = np.diff(df_ecg["Time (ms)"].iloc[peaks])  # Convert peak times to RR intervals

# **Calculate ECG Features**
heart_rate = 60000 / np.mean(rr_intervals) if len(rr_intervals) > 0 else 0
mean_rr = np.mean(rr_intervals) if len(rr_intervals) > 0 else 0
std_rr = np.std(rr_intervals) if len(rr_intervals) > 0 else 0
median_rr = np.median(rr_intervals) if len(rr_intervals) > 0 else 0
energy = np.sum(df_ecg["ECG Signal"]**2)
entropy = -np.sum(df_ecg["ECG Signal"] * np.log(np.abs(df_ecg["ECG Signal"]) + 1e-10))

# **🔹 Plot ECG Signal with R-Peaks**
plt.figure(figsize=(10, 5))
plt.plot(df_ecg["Time (ms)"], df_ecg["ECG Signal"], label="ECG Signal", color='b')
plt.scatter(df_ecg["Time (ms)"].iloc[peaks], df_ecg["ECG Signal"].iloc[peaks], color='r', label="R-Peaks", zorder=3)
plt.xlabel("Time (ms)")
plt.ylabel("ECG Signal")
plt.title("ECG Signal with R-Peaks (After Outlier Removal)")
plt.legend()
plt.grid(True)
plt.show()

# **🔹 Display Extracted Features**
ecg_features = {
    "Heart Rate (BPM)": heart_rate,
    "Mean RR Interval (ms)": mean_rr,
    "Median RR Interval (ms)": median_rr,
    "RR Interval Std Dev (ms)": std_rr,
    "Signal Energy": energy,
    "Signal Entropy": entropy
}

for key, value in ecg_features.items():
    print(f"{key}: {value:.2f}")


Enter lower cutoff frequency for bandpass filter (e.g., 0.5):  0.2
